In [1]:
from models.resnet import *
from Stack_LIME import *
from coordconv import *

import glob

from IPython.display import clear_output

from pytorch_lightning import LightningModule, Trainer

import torch
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader, random_split

import torchvision
from torchvision import transforms
from torchvision.datasets import CIFAR10

import torchmetrics

torch.manual_seed(17)
tf = transforms.ToPILImage()

In [2]:
def create_folder(name):
    try:
        if not os.path.exists(name):
            os.makedirs(name)
    except OSError:
        print ('Error: Creating directory. ' + name)

In [3]:
class UnNormalize(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        """
        Args:
            tensor (Tensor): Tensor image of size (C, H, W) to be normalized.
        Returns:
            Tensor: Normalized image.
        """
        for t, m, s in zip(tensor, self.mean, self.std):
            t.mul_(s).add_(m)
            # The normalize code -> t.sub_(m).div_(s)
        return tensor

In [4]:
class CNN(LightningModule):
    def __init__(self, batch_size = 200, num_workers = 0, train_dataset = None,
    test_dataset = None):
        super().__init__()
        self.model = resnet18(pretrained=False, progress=True, device="cuda")
#         self.model = torchvision.models.squeezenet1_1()
#         self.model.features[0] = nn.Sequential(CoordConv2d(3, 32, 1, with_r=True),nn.Conv2d(32,64,3))
#         self.model.classifier[1] = nn.Conv2d(512, 10, 1, 1)
        self.num_workers = num_workers
        self.batch_size = batch_size
        self.train_dataset = train_dataset
        self.test_dataset = test_dataset
        self.metrics = torchmetrics.Accuracy()
        
    def restart(self, batch_size = 200, num_workers = 0, train_dataset = None,test_dataset = None):
        self.num_workers = num_workers
        self.batch_size = batch_size
        self.train_dataset = train_dataset
        self.test_dataset = test_dataset
        self.metrics = torchmetrics.Accuracy()
        
    def forward(self, x):
        x = self.model(x)
        return x

    def training_step(self, batch, batch_nb):
        x, y = batch
        loss = F.cross_entropy(self(x), y)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        acc = self.metrics(logits, y)
        metrics = {"val_acc": acc, "val_loss": loss}
        self.log_dict(metrics)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        acc = self.metrics(logits, y)
        metrics = {"test_acc": acc, "test_loss": loss}
        self.log_dict(metrics)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.1)

    def setup(self, stage = None):
        if stage == 'fit' or stage is None:
            dataset = self.train_dataset
            dataset_size = dataset.__len__()
            self.train_ds, self.val_ds = random_split(dataset, [int(dataset_size * 0.9) , int(dataset_size * 0.1)])


        if stage == 'test' or stage is None:
            self.test_ds = self.test_dataset

    def train_dataloader(self):
        return DataLoader(self.train_ds, batch_size=self.batch_size, num_workers = self.num_workers, pin_memory=True)

    def val_dataloader(self):
        return DataLoader(self.val_ds, batch_size=self.batch_size, num_workers = self.num_workers, pin_memory=True)

    def test_dataloader(self):
        return DataLoader(self.test_ds, batch_size=self.batch_size, num_workers = self.num_workers, pin_memory=True)

In [5]:
class CIFAR10_edit(Dataset):
    def __init__(self, img_dir, labels, transform = None):
        self.img_dir = img_dir
        self.img_labels = glob.glob(img_dir+'/*')
        self.img = []
        for k in self.img_labels:
            imgs = glob.glob(k + '/*')
            self.img += imgs
        self.transform = transform
        self.labels = labels

    def __len__(self):
        return len(self.img)

    def __getitem__(self, idx):
        img_path = self.img[idx]
        
        image = Image.open(img_path)
        image = np.array(image)

        label = img_path.split('/')
        label = label[2].split('\\')[1]
        label = self.labels[label]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label


In [6]:
transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize((32,32)),
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
    ])
unorm = UnNormalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))

In [7]:
labels = ['airplane', 'automobile', 'bird','cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# train_data = CIFAR10_edit('./data/edit_img_b', labels = labels, transform = transform)
train_data = CIFAR10('./data/cifar10', train = True, download = False, transform = transform)

test_data = CIFAR10('./data/cifar10', train = False, download = False, transform = transform)

In [8]:
model = CNN(batch_size = 512, num_workers = 0 ,
            train_dataset = train_data, test_dataset = test_data)

In [9]:
chk_path = "./model/resnet18_orig_split.ckpt"
model = model.load_from_checkpoint(chk_path)
model.restart(batch_size = 512, num_workers = 8 ,
            train_dataset = train_data, test_dataset = test_data)

In [10]:
trainer = Trainer(accelerator = 'gpu', devices = 1 , max_epochs = 200)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [11]:
model.setup(None)
trainer.test(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

c:\users\jh\anaconda3\envs\ispin\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.37299999594688416, 'test_loss': 7.9693827629089355}
--------------------------------------------------------------------------------


[{'test_acc': 0.37299999594688416, 'test_loss': 7.9693827629089355}]

In [13]:
model.setup('fit')
trainer.validate(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validating: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{'val_acc': 0.9240000247955322, 'val_loss': 0.2444816678762436}
--------------------------------------------------------------------------------


[{'val_acc': 0.9240000247955322, 'val_loss': 0.2444816678762436}]

In [ ]:
a = torch.tensor([[1,2,3,4,5]])
# a.append(torch.tensor([1,2,3,6,7]))
# a.append(torch.tensor([5,3,4,2,4]))
print(a)
c = torch.stack(a, dim =1)
# c = torch.stack([c,d], dim = 0)
c

In [ ]:
x, y = train_data[0]

In [ ]:
y

In [ ]:
from pytorch_lightning.loggers import WandbLogger
